# Unitarly Simulated Kraus

In [396]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random

from src import *

import pickle
from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.circuit.random import random_circuit
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import random_unitary, DensityMatrix, partial_trace


np.set_printoptions(precision=3)

## Choi Form 

In [368]:
n = 1
m = 1
backend = QasmSimulator()

anc_reg = qk.QuantumRegister(n)
rank_reg = qk.QuantumRegister(n)
state_reg = qk.QuantumRegister(m)
clas_reg = qk.ClassicalRegister(n)

random.seed(42)
np.random.seed(42)

U = random_unitary(2**(m+n), seed = 42)
circuit = qk.QuantumCircuit(anc_reg, rank_reg, state_reg, clas_reg)
circuit.h(anc_reg)
circuit.cx(anc_reg, rank_reg)


circuit.append(U, [1, 2])
circuit.save_density_matrix(qubits = [2, 1])

job = qk.execute(circuit, backend, seed = 42, transpiler_seed = 42)
density = job.result().data()['density_matrix'].data
print(density)
density = density[:2, :2]/np.trace(density[:2, :2])
print(density)
print(circuit)



[[ 0.29789732+0.j          0.09618196+0.1026449j   0.00205687-0.09254756j
  -0.13295524-0.11904842j]
 [ 0.09618196-0.1026449j   0.09787566+0.j          0.0392728 +0.06152911j
  -0.11932904+0.00190739j]
 [ 0.00205687+0.09254756j  0.0392728 -0.06152911j  0.45656105+0.j
  -0.05924704+0.04924282j]
 [-0.13295524+0.11904842j -0.11932904-0.00190739j -0.05924704-0.04924282j
   0.14766597+0.j        ]]
[[0.75269746+0.j         0.24302306+0.25935297j]
 [0.24302306-0.25935297j 0.24730254+0.j        ]]
        ┌───┐                    
 q3399: ┤ H ├──■─────────────────
        └───┘┌─┴─┐┌──────────┐ ░ 
 q3400: ─────┤ X ├┤0         ├─░─
             └───┘│  Unitary │ ░ 
 q3401: ──────────┤1         ├─░─
                  └──────────┘ ░ 
c250: 1/═════════════════════════
                                 


In [364]:
U_ = U.data
d = 2**m
rank = 2**n

kraus_list = [U_[:rank, d*i:d*(i+1)] for i in range(rank)]

In [365]:
state = np.array([[1, 0], [0, 0]])
state = sum([K@state@K.T.conj() for K in kraus_list])

In [366]:
print(state)

[[ 0.17186137+0.j         -0.13783648-0.09700634j]
 [-0.13783648+0.09700634j  0.89168424+0.j        ]]


## Manual

In [404]:
n = 1
m = 1

d = 2**m
rank = 2**n
U = random_unitary(2**(m+n), seed = 42).data


kraus_list = [U[:d, d*i:d*(i+1)] for i in range(rank)]
state_init = np.zeros((2**m,  2**m))
state_init[0, 0] = 1
state = sum([K@state_init@K.T.conj() for K in kraus_list])
print(state)

[[ 0.172+0.j    -0.138-0.097j]
 [-0.138+0.097j  0.892+0.j   ]]


In [407]:
state_init = np.zeros((2**m,2**m))
state_init[0, 0] = 1

state = 1/2**n*np.eye(2**n)
state = np.kron(state, state_init)
print(state)

state = U@state@U.T.conj()
print(state)
state = state[:2**m, :2**m]/np.trace(state[:2**m, :2**m])
print(state)

[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  0. ]]
[[ 0.086+0.j    -0.069-0.049j  0.013+0.079j  0.111+0.098j]
 [-0.069+0.049j  0.446+0.j     0.044-0.102j  0.034+0.06j ]
 [ 0.013-0.079j  0.044+0.102j  0.109+0.j     0.148-0.044j]
 [ 0.111-0.098j  0.034-0.06j   0.148+0.044j  0.36 +0.j   ]]
[[ 0.162+0.j    -0.13 -0.091j]
 [-0.13 +0.091j  0.838+0.j   ]]
